# Car Prices – Pandas Assignment (Jupyter Notebook)

> **Note:** Keep this notebook in the **same folder** as `car_prices.csv` so that the code can read the file directly.
> The code tries to automatically detect the correct column names (price, brand, year, odometer, etc.)  
> If your dataset uses slightly different column names, just adjust the `candidate_columns` dictionary once.


## 1. Data Ingestion & Quality Profiling

### 1.1 Load & Inspect

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Display plots inside the notebook
%matplotlib inline

# 1.1 – Read the CSV file
df = pd.read_csv("car_prices.csv")

print("First 5 rows:")
display(df.head())

print("\nBasic info:")
print(df.info())

print("\nTotal number of records:", len(df))


### 1.2 Understanding the Data Structure  

We will also create a small helper that maps the **logical names** we need (price, brand, year, etc.)  
to the **actual column names** present in your CSV.


In [ ]:
print("Shape of the dataset (rows, columns):", df.shape)

print("\nColumn names:")
print(df.columns.tolist())

print("\nData types:")
print(df.dtypes)

# --- Helper: detect important columns automatically ---
candidate_columns = {
    "price": ["sellingprice", "selling_price", "price", "Selling_Price", "sellingPrice"],
    "brand": ["make", "brand", "Brand", "Make"],
    "model": ["model", "Model", "model_name", "Model_Name"],
    "year": ["year", "model_year", "Year", "Model_Year"],
    "odometer": ["odometer", "mileage", "Odometer", "Mileage"],
    "condition": ["condition", "Condition", "condition_score", "Condition_Score"],
    "state": ["state", "State", "state_or_province"],
    "color": ["color", "Color", "exterior_color", "Exterior_Color"],
    "interior": ["interior", "Interior", "interior_color", "Interior_Color"],
}

col_map = {}

for logical_name, candidates in candidate_columns.items():
    for c in candidates:
        if c in df.columns:
            col_map[logical_name] = c
            break

print("\nDetected column mapping (logical_name -> actual column):")
for k, v in col_map.items():
    print(f"{k:10s} -> {v}")

# Optional: you can manually adjust any incorrect mapping here, e.g.
# col_map["price"] = "selling_price"


### 1.3 Missing & Anomaly Detection

In [ ]:
# 1.3.1 – Quantify nulls per column
null_counts = df.isna().sum()
null_percent = (null_counts / len(df)) * 100

print("Null values per column:")
display(pd.DataFrame({
    "null_count": null_counts,
    "null_percent": null_percent.round(2)
}))

# Visualize missing values using a simple bar chart
plt.figure(figsize=(10, 4))
plt.bar(null_counts.index, null_counts.values)
plt.xticks(rotation=90)
plt.ylabel("Number of missing values")
plt.title("Missing values per column")
plt.tight_layout()
plt.show()

# 1.3.2 – Resolve null values
# Strategy:
# - Numeric columns: fill with median
# - Categorical columns: fill with mode (most frequent)
numeric_cols = df.select_dtypes(include=[np.number]).columns
categorical_cols = df.select_dtypes(include=["object", "category"]).columns

for col in numeric_cols:
    if df[col].isna().sum() > 0:
        median_val = df[col].median()
        df[col].fillna(median_val, inplace=True)

for col in categorical_cols:
    if df[col].isna().sum() > 0:
        mode_val = df[col].mode()[0]
        df[col].fillna(mode_val, inplace=True)

print("\nNull values after imputation:")
print(df.isna().sum())

# 1.3.3 – Duplicates
dup_count = df.duplicated().sum()
print("\nNumber of duplicate records:", dup_count)

if dup_count > 0:
    df = df.drop_duplicates().reset_index(drop=True)
    print("Duplicates removed. New shape:", df.shape)


## 2. DataFrame Queries

### 2.1 Calculate the average, minimum, and maximum car price

In [ ]:
price_col = col_map.get("price")
if price_col is None:
    raise KeyError("Price column not detected. Please set col_map['price'] manually.")

avg_price = df[price_col].mean()
min_price = df[price_col].min()
max_price = df[price_col].max()

print(f"Average price: {avg_price:,.2f}")
print(f"Minimum price: {min_price:,.2f}")
print(f"Maximum price: {max_price:,.2f}")


### 2.2 List all unique colors of cars

In [ ]:
color_col = col_map.get("color")
if color_col is None:
    print("No color column detected. Please set col_map['color'] manually.")
else:
    unique_colors = df[color_col].dropna().unique()
    print("Unique colors:")
    print(unique_colors)


### 2.3 Find the number of unique car brands and car models

In [ ]:
brand_col = col_map.get("brand")
model_col = col_map.get("model")

if brand_col is None or model_col is None:
    print("Please make sure both 'brand' and 'model' columns are correctly mapped in col_map.")
else:
    n_brands = df[brand_col].nunique()
    n_models = df[model_col].nunique()

    print(f"Number of unique car brands: {n_brands}")
    print(f"Number of unique car models: {n_models}")


### 2.4 Find all car information having selling prices greater than $165,000

In [ ]:
if price_col is None:
    price_col = col_map.get("price")

high_price_cars = df[df[price_col] > 165000]
print(f"Total cars with price > 165000: {len(high_price_cars)}")
display(high_price_cars.head())


### 2.5 Find the top 5 most frequently sold car models

In [ ]:
if model_col is None:
    model_col = col_map.get("model")

top_models = df[model_col].value_counts().head(5)
print("Top 5 most frequently sold car models:")
display(top_models)


### 2.6 What is the average selling price of cars by brand (make)?

In [ ]:
if brand_col is None:
    brand_col = col_map.get("brand")

avg_price_by_brand = df.groupby(brand_col)[price_col].mean().sort_values(ascending=False)
print("Average selling price by brand (make):")
display(avg_price_by_brand)


### 2.7 What is the minimum selling price of cars for each interior?

In [ ]:
interior_col = col_map.get("interior")

if interior_col is None:
    print("No interior column detected. Please set col_map['interior'] manually if available.")
else:
    min_price_by_interior = df.groupby(interior_col)[price_col].min().sort_values()
    print("Minimum selling price by interior:")
    display(min_price_by_interior)


### 2.8 Find highest odometer reading per year from highest to lowest order

In [ ]:
year_col = col_map.get("year")
odo_col = col_map.get("odometer")

if year_col is None or odo_col is None:
    print("Please ensure 'year' and 'odometer' columns are correctly mapped in col_map.")
else:
    max_odo_per_year = (
        df.groupby(year_col)[odo_col]
        .max()
        .sort_values(ascending=False)
    )
    print("Highest odometer reading per year (sorted from highest to lowest):")
    display(max_odo_per_year)


### 2.9 Create a new column for car age (assuming the current year is 2025)

In [ ]:
if year_col is None:
    year_col = col_map.get("year")

CURRENT_YEAR = 2025
df["car_age"] = CURRENT_YEAR - df[year_col]

print("Sample of car_age column:")
display(df[[year_col, "car_age"]].head())


### 2.10 Find the number of cars having a condition ≥ 48 and odometer > 90,000

In [ ]:
cond_col = col_map.get("condition")
odo_col = col_map.get("odometer", odo_col)

if cond_col is None or odo_col is None:
    print("Please ensure 'condition' and 'odometer' columns are correctly mapped in col_map.")
else:
    mask = (df[cond_col] >= 48) & (df[odo_col] > 90000)
    count_cars = mask.sum()
    print(f"Number of cars with condition >= 48 and odometer > 90000: {count_cars}")


### 2.11 Which state consistently has higher car prices for newer cars (year > 2013)?

In [ ]:
state_col = col_map.get("state")

if state_col is None or year_col is None:
    print("Please ensure 'state' and 'year' columns are correctly mapped in col_map.")
else:
    newer_cars = df[df[year_col] > 2013]
    avg_price_by_state_new = (
        newer_cars.groupby(state_col)[price_col]
        .mean()
        .sort_values(ascending=False)
    )
    print("Average price by state for cars with year > 2013:")
    display(avg_price_by_state_new)

    if len(avg_price_by_state_new) > 0:
        print("\nState with highest average price for newer cars:",
              avg_price_by_state_new.index[0])


### 2.12 For cars with excellent condition (top 20%), which makes have the lowest average price (value for money)?

In [ ]:
if cond_col is None:
    cond_col = col_map.get("condition")

if brand_col is None:
    brand_col = col_map.get("brand")

if cond_col is None or brand_col is None:
    print("Please ensure 'condition' and 'brand' columns are correctly mapped in col_map.")
else:
    threshold = df[cond_col].quantile(0.8)  # top 20%
    excellent_cars = df[df[cond_col] >= threshold]

    value_for_money = (
        excellent_cars.groupby(brand_col)[price_col]
        .mean()
        .sort_values(ascending=True)
    )

    print("Brands with lowest average price among top 20% condition cars (best value for money):")
    display(value_for_money.head(10))


## 3. Data Visualization and Insights

### 3.1 Show the correlation of all numerical features (e.g., selling price, odometer)

In [ ]:
# Select only numeric columns
numeric_df = df.select_dtypes(include=[np.number])

print("Numeric columns used for correlation:")
print(numeric_df.columns.tolist())

corr_matrix = numeric_df.corr()

plt.figure(figsize=(8, 6))
plt.imshow(corr_matrix, aspect='auto')
plt.colorbar(label="Correlation")
plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=90)
plt.yticks(range(len(corr_matrix.index)), corr_matrix.index)
plt.title("Correlation heatmap (numeric features)")
plt.tight_layout()
plt.show()

corr_matrix


### 3.2 Plot average selling price by year and explain the pattern

In [ ]:
avg_price_by_year = df.groupby(year_col)[price_col].mean().reset_index()

plt.figure(figsize=(8, 5))
plt.bar(avg_price_by_year[year_col], avg_price_by_year[price_col])
plt.xlabel("Year")
plt.ylabel("Average Selling Price")
plt.title("Average Selling Price by Year")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

avg_price_by_year


### 3.3 Plot a graph to show average selling price by odometer

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(df[odo_col], df[price_col], alpha=0.3)
plt.xlabel("Odometer Reading")
plt.ylabel("Selling Price")
plt.title("Selling Price vs Odometer")
plt.tight_layout()
plt.show()


### 3.4 Plot number of cars sold in each state and find the top 3 highest car selling states

In [ ]:
if state_col is None:
    state_col = col_map.get("state")

if state_col is None:
    print("State column not detected. Please set col_map['state'].")
else:
    cars_per_state = df[state_col].value_counts()

    plt.figure(figsize=(10, 5))
    plt.bar(cars_per_state.index, cars_per_state.values)
    plt.xlabel("State")
    plt.ylabel("Number of cars sold")
    plt.title("Number of cars sold in each state")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

    print("Top 3 highest car selling states:")
    display(cars_per_state.head(3))


### 3.5 Plot a bar graph of average selling price by condition score ranges of size 5

In [ ]:
cond_col = col_map.get("condition", cond_col)

if cond_col is None:
    print("Condition column not detected. Please set col_map['condition'].")
else:
    cond_series = df[cond_col]
    min_c = int(cond_series.min())
    max_c = int(cond_series.max())

    # Create bins of size 5
    bins_5 = list(range((min_c // 5) * 5, ((max_c // 5) + 1) * 5 + 1, 5))
    df["condition_bin_5"] = pd.cut(cond_series, bins=bins_5, include_lowest=True)

    avg_price_by_cond_bin = df.groupby("condition_bin_5")[price_col].mean()

    plt.figure(figsize=(10, 5))
    plt.bar(avg_price_by_cond_bin.index.astype(str), avg_price_by_cond_bin.values)
    plt.xlabel("Condition score range (size 5)")
    plt.ylabel("Average selling price")
    plt.title("Average selling price by condition score ranges of size 5")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    avg_price_by_cond_bin


### 3.6 Plot a bar graph of number of cars sold by condition ranges of size 10

In [ ]:
if cond_col is None:
    cond_col = col_map.get("condition")

if cond_col is None:
    print("Condition column not detected. Please set col_map['condition'].")
else:
    cond_series = df[cond_col]
    min_c = int(cond_series.min())
    max_c = int(cond_series.max())

    # Create bins of size 10
    bins_10 = list(range((min_c // 10) * 10, ((max_c // 10) + 1) * 10 + 1, 10))
    df["condition_bin_10"] = pd.cut(cond_series, bins=bins_10, include_lowest=True)

    count_by_cond_bin = df["condition_bin_10"].value_counts().sort_index()

    plt.figure(figsize=(10, 5))
    plt.bar(count_by_cond_bin.index.astype(str), count_by_cond_bin.values)
    plt.xlabel("Condition score range (size 10)")
    plt.ylabel("Number of cars")
    plt.title("Number of cars sold by condition score ranges of size 10")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    count_by_cond_bin


### 3.7 Box plot of car selling prices grouped by color (with and without outliers)

In [ ]:
color_col = col_map.get("color", color_col)

if color_col is None:
    print("Color column not detected. Please set col_map['color'].")
else:
    # Boxplot with all data (including outliers)
    colors = df[color_col].unique()

    plt.figure(figsize=(12, 6))
    data = [df[df[color_col] == c][price_col] for c in colors]
    plt.boxplot(data, labels=colors, showfliers=True)
    plt.xlabel("Color")
    plt.ylabel("Selling Price")
    plt.title("Selling price distribution by color (with outliers)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Remove outliers using IQR on price
    Q1 = df[price_col].quantile(0.25)
    Q3 = df[price_col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_no_outliers = df[(df[price_col] >= lower_bound) & (df[price_col] <= upper_bound)]

    colors_no = df_no_outliers[color_col].unique()
    plt.figure(figsize=(12, 6))
    data_no = [df_no_outliers[df_no_outliers[color_col] == c][price_col] for c in colors_no]
    plt.boxplot(data_no, labels=colors_no, showfliers=False)
    plt.xlabel("Color")
    plt.ylabel("Selling Price")
    plt.title("Selling price distribution by color (outliers removed)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
